## Guider mode quick look
This is intended to unpack the rawStamps from guider mode FITS files and 
give a quick look of all 16 stamps for all 4 CCDs. \
Craig Lage - 15-Apr-25

In [ ]:
import os
import shlex, subprocess
import matplotlib.pyplot as plt
from lsst.summit.utils.plotting import plot
import numpy as np
from astropy.io import fits
from lsst.resources import ResourcePath
from lsst.afw import cameraGeom
from lsst.obs.lsst.cameraTransforms import LsstCameraTransforms
from lsst.obs.lsst import LsstCam
from matplotlib.backends.backend_pdf import PdfPages
from photutils.detection import DAOStarFinder
import pickle as pkl
from astropy.time import Time, TimeDelta

# Get the main header and the information it contains

In [ ]:
def getMainHeaderInfo(hdu_list):
    hdr0 = hdu_list[0].header
    roiCol = hdr0['ROICOL']
    roiRow = hdr0['ROIROW']
    roiCols = hdr0['ROICOLS']
    roiRows = hdr0['ROIROWS']
    try:
        roiUnder = hdr0['ROIUNDRC']
    except:
        roiUnder = 6
    nStamps = hdr0['N_STAMPS']
    
    # Set the xor value - Guider CCDs are different from science CCDs
    if raft in ['R00', 'R04', 'R40', 'R44']:
        # Guider rafts
        xor = 0x20000
    else:
        # Science rafts
        xor = 0x1ffff
    return [roiRow, roiCol, roiRows, roiCols, roiUnder, nStamps, xor]

# Now define the code to unpack the rawStamps:

In [ ]:
def unpackStamps(hduNum):
    data = np.array(hdu_list[hduNum].data[0]).astype('>u4')[0]
    data.byteswap(inplace=True)
    totalCols = roiCols + roiUnder
    size = roiRows * totalCols
    out = np.zeros([16, size], dtype=int)
    image_out = np.zeros([16, roiRows, roiCols], dtype=int)
    for n in range(size):
        # Get 9 32 bit words of data
        res = ''
        for i in range(9):
            d = data[(size - n) * 9 - i - 1]
            d = format(d, '#034b')
            d = d.split('b')[1]
            res += d
        # Now extract 16 18 bit words from the data
        for i in range(16):
            bin_value = res[i * 18:(i + 1) * 18]
            int_value = int(bin_value, 2)
            final_value = int_value ^ xor
            out[i,n] = final_value  
    for i in range(16):
        reshaped = out[i,:].reshape(roiRows, totalCols)
        image_out[i,:,:] = np.flipud(np.fliplr(reshaped[:,0:roiCols]))
    return image_out

# The below converts the amp name into the sequential numbering that comes
# from unpacking.
ampDict = {'C00':0, 'C01':1, 'C02':2, 'C03':3, 'C04':4,
           'C05':5, 'C06':6, 'C07':7, 'C10':8, 'C11':9,
           'C12':10, 'C13':11, 'C14':12, 'C15':13, 'C16':14,
           'C17':15}

# Scroll through all frames, logging the locations of the stars
# Only frame 0 is plotted in the Guider_Mode_CCDs_{expId} plot.

In [ ]:
camera = LsstCam.getCamera()

dayObs = 20250417
seqNum = 599
#n = 2 # This just unpacks one frame.  This chooses which frame
expId = int(f"{dayObs}{seqNum:05d}")
detectors = [['R00', 'SG0'], ['R00', 'SG1']]

"""
detectors = [['R00', 'SG0'], ['R00', 'SG1'], 
          ['R04', 'SG0'], ['R04', 'SG1'], 
          ['R40', 'SG0'], ['R40', 'SG1'],
          ['R44', 'SG0'], ['R44', 'SG1']]
"""
pdf = PdfPages(f"/home/c/cslage/u/Guider_Mode/Guider_Mode_CCDs_R00_{expId}.pdf")
fig = plt.figure(figsize=(10,10))
motionDict = {}
edgeSkip = 10 # pixels to keep away from the edge
xyTol = 4.0 # Tolerance for star motion in pixels

# Get one CCD to know nStamps
raft = 'R00'
ccd = 'SG0'
filename = f"s3://embargo@rubin-summit/LSSTCam/{dayObs}/MC_O_{dayObs}_{seqNum:06d}/MC_O_{dayObs}_{seqNum:06d}_{raft}_{ccd}_guider.fits"

rp = ResourcePath(filename)
with rp.open(mode="rb") as f:
    hdu_list = fits.open(f)
[roiRow, roiCol, roiRows, roiCols, roiUnder, nStamps, xor] = getMainHeaderInfo(hdu_list)
llX = roiCol
llY = roiRow
urX = roiCol + roiCols
urY = roiRow + roiRows

for n in range(1, nStamps+1):

    for [raft, ccd] in detectors:
        detName = f"{raft}_{ccd}"
        filename = f"s3://embargo@rubin-summit/LSSTCam/{dayObs}/MC_O_{dayObs}_{seqNum:06d}/MC_O_{dayObs}_{seqNum:06d}_{raft}_{ccd}_guider.fits"
        rp = ResourcePath(filename)
        with rp.open(mode="rb") as f:
            hdu_list = fits.open(f)
        [roiRow, roiCol, roiRows, roiCols, roiUnder, nStamps, xor] = getMainHeaderInfo(hdu_list)
        
        for detector in camera:
            if detector.getName()== detName:
                break
        bbox = detector.getBBox()
        nx,ny = bbox.getDimensions()
        fullImage = np.zeros((ny,nx))
        #fullImage[:,:]=np.nan
    
        hduNum = 2 * n + 1
        hdrn = hdu_list[hduNum].header
        timestamp = hdrn['STMPTIME']
        image_out = unpackStamps(hduNum)
        lct = LsstCameraTransforms(camera,detName)
        cornerDict = {}
        #print(raft, ccd)
        for amp in detector.getAmplifiers():
            ampName = amp.getName()
            seg = ampDict[ampName]
    
            llCCDX,llCCDY = lct.ampPixelToCcdPixel(llX,llY,ampName)  # get CCD x,y these are floats
            llfpX,llfpY = lct.ampPixelToFocalMm(llX,llY,ampName)     # get focal plane x,y in mm
    
            urCCDX,urCCDY = lct.ampPixelToCcdPixel(urX,urY,ampName)
            urfpX,urfpY = lct.ampPixelToFocalMm(urX,urY,ampName)
            edgeSkip = 20 # pixels
            cornerDict[ampName] = []
            cornerDict[ampName].append(min(llCCDX,urCCDX) + edgeSkip)
            cornerDict[ampName].append(min(llCCDY,urCCDY) + edgeSkip)
            cornerDict[ampName].append(max(llCCDX,urCCDX) - edgeSkip)
            cornerDict[ampName].append(max(llCCDY,urCCDY) - edgeSkip)
        
            roiarr = image_out[seg]
            roiarr = roiarr - np.median(roiarr)
            """
            if urCCDX < llCCDX:
                roiarr = np.fliplr(roiarr)
            if urCCDY < llCCDY:
                roiarr = np.flipud(roiarr)
            """
            
            # this doesn't workout how to map the stamps to the CCD, but the ll,ur points have enough info
            fullImage[int(min(llCCDY,urCCDY)):int(max(llCCDY,urCCDY)),
                int(min(llCCDX,urCCDX)):int(max(llCCDX,urCCDX))] = roiarr 

        if n == 1:
            x = plot(fullImage, stretch='ccs', figure=fig)
        daofind = DAOStarFinder(fwhm=1.5, threshold = 500.0)
        sources = daofind(fullImage)
        if sources:
            for i in range(len(sources)):
                xSource = sources['xcentroid'][i]
                ySource = sources['ycentroid'][i]
                for amp in detector.getAmplifiers():
                    ampName = amp.getName()
                    (llShrinkX, llShrinkY, urShrinkX, urShrinkY) = cornerDict[ampName]
                    #print(ampName, xSource, llShrinkX, urShrinkX)
                    #print(ampName, ySource, llShrinkY, urShrinkY)
                    if (xSource > llShrinkX) and (xSource < urShrinkX) and \
                    (ySource > llShrinkY) and (ySource < urShrinkY):
                        fpX, fpY = lct.ccdPixelToFocalMm(xSource, ySource, detName)
                        if n == 1:
                            axs = fig.get_axes()
                            axs[0].set_title(f"Guider mode {expId} {raft}_{ccd}, \n Frame {n} {timestamp}", fontsize=18) 
                            circ = plt.Circle((xSource, ySource), radius=50, color='yellow', fill=False)
                            axs[0].add_artist(circ)
                            # Create initial keys
                            motionKey = f"{raft}_{ccd}_{ampName}_{int(xSource)}_{int(ySource)}"
                            print(n, motionKey)
                            motionDict[motionKey] = []
                            motionDict[motionKey].append([timestamp, (fpX, fpY)])
                        else:
                            for key in motionDict.keys():
                                # Put data in the appropriate key
                                xSTest = float(key.split('_')[3])
                                ySTest = float(key.split('_')[4])
                                if (raft == key.split('_')[0]) and \
                                (ccd == key.split('_')[1]) and \
                                (ampName == key.split('_')[2]) and \
                                (abs(xSource - xSTest) < xyTol) and \
                                (abs(ySource - ySTest) < xyTol):
                                    print(f"Found {key} for frame {n}")
                                    motionDict[key].append([timestamp, (fpX, fpY)])
                        #print(detName, ampName, fpX, fpY)
        if n == 1:
            pdf.savefig(fig)
            #break
            plt.clf()
        print(f"Finished {raft}_{ccd} for frame {n}")
pdf.close()

outfile = open('/home/c/cslage/u/Guider_Mode/MotionDict_R00_22Apr25.pkl', 'wb')
pkl.dump(motionDict,outfile)
outfile.close()

In [ ]:
lct.ampPixelToCcdPixel(llX,llY,ampName)

# To read the data from the dictionary and plot

In [ ]:
infile = open('/home/c/cslage/u/Guider_Mode/MotionDict_19Apr25.pkl', 'rb')
motionDict = pkl.load(infile)
infile.close()

dayObs = 20250417
seqNum = 599
expId = int(f"{dayObs}{seqNum:05d}")

pdf = PdfPages(f"/home/c/cslage/u/Guider_Mode/Guider_Mode_Star_Motion_{expId}.pdf")
fig = plt.figure(figsize=(10,10))
for key in motionDict.keys():
    ax = fig.subplots(1,1)
    print(key)
    data = motionDict[key]
    ts = []
    xs = []
    ys = []
    for [t, (x, y)] in data:
        ts.append(Time(t, scale='utc').unix_tai)
        xs.append(x)
        ys.append(y)
    ts = np.array(ts)
    xs = np.array(xs)
    ys = np.array(ys)
    ts -= ts[0]
    xs = (xs - np.median(xs)) * 100.0 * 0.2
    ys = (ys - np.median(ys)) * 100.0 * 0.2
    ax.plot(ts, xs, color='blue', marker='x', label='X')
    ax.plot(ts, ys, color='red', marker='+', label='Y')
    ax.set_xlabel('Time (seconds)')
    ax.set_ylabel('Median subtracted position (arcsec)')
    ax.set_title(f"{expId} {key} {data[0][0]}")
    ax.legend()
    pdf.savefig(fig)
    plt.clf()
pdf.close()



In [ ]:
jumps = [['R00_SG0', +1], 
 ['R00_SG1', -1],
 ['R04_SG0', 0],
 ['R04_SG1', -1],
 ['R40_SG0', 0],
 ['R40_SG1', 0],
 ['R44_SG0', +1],
 ['R44_SG1', -1]]